<a href="https://colab.research.google.com/github/gbwiersum/gbwiersum/blob/main/RagBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_ollama import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.prompts.chat import ChatPromptTemplate
import streamlit as st
import ollama

base_url = "localhost:11434/"
modlist = [m["model"] for m in ollama.list()["models"]]
modelname = modlist[0]
response={"answer":""}

def start_llm(modelname, base_url='localhost:11434', temperature=0.2, num_predict=512, top_k=50, top_p=50):
    llm = ChatOllama(model=modelname,
                    base_url=base_url,
                    temperature=temperature,
                    num_predict=num_predict,
                    top_k=top_k,
                    top_p=top_p
                        )
    return llm

sysprompt = """You are a generative AI assistant designed to help
researchers working in national defense.
Your answers must work step-by-step to build an analysis synthesizing
from the information provided. Above all, you're constructing an argument.
The goal is to return the logical steps and information sources used
to answer the question. If you don't have sufficient information reply
"I don't know".""".replace("\n", " ").replace("  ", " ")


prompt = ChatPromptTemplate([("system", sysprompt),
                            ("user", "{question}"),
                            ("system", "{context}"),
                            "assistant:"])

embeddings = OllamaEmbeddings(model = modelname, base_url=base_url)
vector_store = InMemoryVectorStore.from_documents("", embeddings)

ModuleNotFoundError: No module named 'langchain_ollama'

In [ ]:
# Document Loaders for working with BytesIO that Streamlit generates:
# These should go into RagBot definitions or maybe a utils?
from io import BytesIO
from langchain_community.document_loaders.parsers.pdf import PyPDFParser
from langchain_core.document_loaders.base import BaseLoader
from langchain_core.document_loaders.blob_loaders import Blob
from langchain_core.documents import Document

class RagBot:
    def __init__(self, llm, vector_store):
        self.llm = llm
        self.vector_store = vector_store

    class CustomPDFLoader(BaseLoader):
        def __init__(self, stream: BytesIO, password =None, extract_images: bool = False):
            self.stream = stream
            self.parser = PyPDFParser(password=password, extract_images=extract_images)

        def load(self) -> list[Document]:
            blob = Blob.from_data(self.stream.getvalue())
            return list(self.parser.parse(blob))

    def add_vector(pages:list[Document], chunk_size:int =1000, chunk_overlap: int =200, vector_store=vector_store):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        all_splits = text_splitter.split_documents(pages)
        vector_store.add_documents(all_splits)

    def pdf_from_path(self, filepath):
        loader = PyPDFLoader(filepath)
        pages = [p for p in loader.lazy_load()]
        self.add_vector(pages)

    def pdf_from_bytes(self, uploaded: list[list, bytes]) -> Document:
        if type(uploaded)==list:
            for u in uploaded:
                cpl = self.CustomPDFLoader(u)
                pages = [p for p in cpl.lazy_load()]
                self.add_vector(pages)
        else:
            cpl = self.CustomPDFLoader(uploaded)
            pages = [p for p in cpl.lazy_load()]
            self.add_vector(pages)


    # Define state for application
    class State(TypedDict):
        question: str
        context: List[Document]
        answer: str

    # Define application steps
    def retrieve(state: State):
        retrieved_docs = vector_store.similarity_search(state["question"])
        return {"context": retrieved_docs}


    def generate(self, state: State):
        docs_content = "\n\n".join(doc.page_content for doc in state["context"])
        messages = prompt.invoke({"question": state["question"], "context": docs_content})
        response = self.llm.invoke(messages)
        return {"answer": response.content}

    def run_query(self, question: str):
        retrieved_docs = vector_store.similarity_search(question)
        got = {"context": retrieved_docs}
        docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
        messages = prompt.invoke({"question": question, "context": docs_content})
        response = self.llm.invoke(messages)
        return response, got

In [ ]:
llm = start_llm(modlist[0], temperature=0.8, num_predict=1028, top_k=100, top_p=100)
Bot = RagBot(llm, vector_store)

def add_folder(folder):
    for f in [p for p in os.walk(folder)][0][2]:
        print("adding: {f}")
        pdf_from_path(folder+"/"+f)

add_folder("/mnt/c/Users/gwiersum/Reference")

In [ ]:
ex = Bot.run_query("What is the meaning of life, the universe and everything?")
print(ex[0].content)
print([i.metadata for i in ex[1]["context"]])